# Поиск мероприятий, проведенных в отношении земельных участков, расположенных в ЗОУИТ

**Исходные данные:** таблицы, содержащие информацию по проведенным за 2022, 2023 гг. проверочными мероприятиям, и таблица с земельными участками, расположенными в границах зоны с особыми условиями использования территории (далее - ЗОУИТ).

**Задача:** найти проверочные мероприятия, которые проводились на земельных участках, расположенных в ЗОУИТ.

В таблицах с мероприятиями отдельный столбец с кадастровыми номерами участков отсутствует. Но есть столбец с кадастровыми номерами одного или нескольких участков вместе с адресом.

## Загрузка и обработка данных

In [1]:
import pandas as pd

In [2]:
# проверочные мероприятия за 2022 год
df_2022 = pd.read_csv('df_2022utf_cut.csv', encoding='utf-8', sep=';')

In [3]:
# проверочные мероприятия за 2023 год
df_2023 = pd.read_csv('df_2023utf_cut.csv', encoding='utf-8', sep=';')

In [4]:
# таблица с участками, расположенными в ЗОУИТ
df_zu = pd.read_csv('zu.csv', encoding='utf-8', sep=';')

In [5]:
# функция показывает общую информацию о таблице и наличие дубликатов
def open_f(data):
    print(data.info())
    
    #проверка наличия явных дубликатов
    dup = data.duplicated().sum()
   
    print('\nКоличество дубликатов:', dup)
    print('Процент дубликатов: {:.2%}\n'.format(dup / data.shape[0]))

In [6]:
for df in df_2022, df_2023, df_zu:
    open_f(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158 entries, 0 to 1157
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Ф.И.О. инспектора                     1158 non-null   object
 1   № распоряжения о проведении проверки  1158 non-null   object
 2   Лицо (код)                            1022 non-null   object
 3   Вид проверки (код)                    1095 non-null   object
 4   Адрес ЗУ, кадастровый номер           1158 non-null   object
 5   Площадь земельного участка (кв. м)    1092 non-null   object
 6   Категория земель                      1092 non-null   object
 7   Вид разрешенного использования        1092 non-null   object
 8   Вид права                             957 non-null    object
 9   Результат проверки                    1135 non-null   object
dtypes: object(10)
memory usage: 90.6+ KB
None

Количество дубликатов: 0
Процент дубликатов: 0.00%

<

В таблице `df_zu` 15 строк-дубликатов. Но для целей задачи представляют интерес только кадастровые номера земельных участков.

Удалим дубликаты кадастровых номеров земельных участков.

In [7]:
# удалим дубликаты в столбце с кадастровыми номерами
print('Дубликатов участков:', df_zu.duplicated(subset='Кадастровый номер').sum())
df_zu = df_zu.drop_duplicates(subset='Кадастровый номер').reset_index(drop=True)

Дубликатов участков: 75


Проверим успешность удаления дубликатов.

In [8]:
# все дубликаты
if df_zu.duplicated().sum() == 0:
    print('Дубликаты удалены')
else:
    print('Дубликатов: ', df_zu.duplicated().sum())

# дубликаты кадастровых номеров земельных участков
if df_zu.duplicated(subset='Кадастровый номер').sum() == 0:
    print('Дубликаты участков удалены')
else:
    print('Дубликатов: ', df_zu.duplicated(subset='Кадастровый номер').sum())
    
df_zu.info()

Дубликаты удалены
Дубликаты участков удалены
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Реестровый_номер   1492 non-null   object
 1   Тип_зоны           1492 non-null   object
 2   Наименование       1492 non-null   object
 3   Кадастровый номер  1492 non-null   object
 4   Площадь            1492 non-null   object
 5   Вид ЗУ             1492 non-null   object
 6   Категория          1492 non-null   object
dtypes: object(7)
memory usage: 81.7+ KB


## Поиск мероприятий

Выделим кадастровые номера земельных участков, расположенных в ЗОУИТ.

In [9]:
zu_in_zone = df_zu['Кадастровый номер']

In [10]:
zu_in_zone.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1492 entries, 0 to 1491
Series name: Кадастровый номер
Non-Null Count  Dtype 
--------------  ----- 
1492 non-null   object
dtypes: object(1)
memory usage: 11.8+ KB


Напишем функцию, которая будет искать среди участков, в отношении которых были проведены мероприятия, участки, расположенные в ЗОУИТ.

In [11]:
# на вход подается столбец (ячейка через apply) с адресом и кадастровым номером
# на выходе - True, если кадастровый номер есть в списке участков в зоне, False - если нет. None на всякий случай.

def zu_search(row):
    x = []    # пустой список
    for s in range(zu_in_zone.shape[0]-1):
        if row.find(zu_in_zone[s]) not in x:    # .find возвращает индекс, начиная с которого подстрока встречается в исходной строке
            x.append(row.find(zu_in_zone[s]))
    if len(x) == 1:    # только значение -1, что соответствует отсутствию кадастрового номера в списке участков в зоне
        return False
    elif len(x) > 1:
        return True
    else:
        return None

Проверим мероприятия за 2022 год:

In [12]:
df_2022['in_zone'] = df_2022['Адрес ЗУ, кадастровый номер'].apply(zu_search)

In [13]:
# проверим, есть ли None
len(df_2022[df_2022['in_zone'] == None])

0

In [14]:
# выведем интересующие мероприятия и их количество
#df_2022[df_2022['in_zone'] == True]    # закомментировано в целях сохранения конфедициальности
len(df_2022[df_2022['in_zone'] == True])

2

Проверим мероприятия за 2023 год:

In [15]:
df_2023['in_zone'] = df_2023['Адрес ЗУ, кадастровый номер'].apply(zu_search)

In [16]:
# проверим, есть ли None
len(df_2023[df_2023['in_zone'] == None])

0

In [17]:
# выведем интересующие мероприятия и их количество
#df_2023[df_2023['in_zone'] == True]    # закомментировано в целях сохранения конфедициальности
len(df_2023[df_2023['in_zone'] == True])

0

## Вывод

В отношении земельных участков, расположенных в границах ЗОУИТ, проведено 2 проверочных мероприятия в 2022 г. и 0 мероприятий в 2023 г.